# Dashboard parameter updater

This is a helper file for **developers**.

You run this notebook manually after cloning the repo, and before committing back, to swap the `catalog` and `schema` parameters for any dashboard in `../dashboards/` that follow the `use identifier(:parameter)` syntax.

**Steps:**
- After you have branched the code, you want to develop against your dev version of the dashboard (The file in this repo!)
- Pass your user and catalog, then run this notebook and the files will be updated.
- You're now good to go with developing against the repo copy of the dashboard.
- Before you commit back, set the widget `pre_commit_update` to true 

In [0]:
import json
import os
from typing import Dict, Any


In [0]:
"""
Minimal script to update catalog and schema parameters in Databricks dashboard JSON files.
"""

def update_parameters(dashboard_data: Dict[str, Any], new_catalog: str = None, new_schema: str = None) -> Dict[str, Any]:
    """
    Update catalog, schema parameters in dashboard JSON.
    
    Args:
        dashboard_data: The dashboard JSON data
        new_catalog: New catalog value (None to skip)
        new_schema: New schema value (None to skip)
    
    Returns:
        Updated dashboard data
    """
    if 'datasets' not in dashboard_data:
        return dashboard_data
    
    # Track updates for reporting
    updates = []
    
    # Iterate through all datasets
    for dataset in dashboard_data['datasets']:
        dataset_name = dataset.get('displayName', 'unknown')
        
        if 'parameters' not in dataset:
            continue
        
        # Check each parameter
        for param in dataset['parameters']:
            keyword = param.get('keyword', '')
            
            # Skip if no defaultSelection
            if 'defaultSelection' not in param:
                continue
            
            # Initialize the structure if it's empty or missing required keys
            if not param['defaultSelection'] or 'values' not in param['defaultSelection']:
                param['defaultSelection'] = {
                    'values': {
                        'dataType': param.get('dataType', 'STRING'),
                        'values': [{'value': ''}]
                    }
                }
            elif 'values' not in param['defaultSelection']['values']:
                param['defaultSelection']['values']['values'] = [{'value': ''}]
            elif len(param['defaultSelection']['values']['values']) == 0:
                param['defaultSelection']['values']['values'] = [{'value': ''}]
            
            # Update catalog parameter
            if keyword == 'catalog' and new_catalog is not None:
                old_value = param['defaultSelection']['values']['values'][0]['value']
                param['defaultSelection']['values']['values'][0]['value'] = new_catalog
                updates.append(f"Dataset '{dataset_name}': catalog '{old_value}' -> '{new_catalog}'")
            
            # Update schema parameter
            elif keyword == 'schema' and new_schema is not None:
                old_value = param['defaultSelection']['values']['values'][0]['value']
                param['defaultSelection']['values']['values'][0]['value'] = new_schema
                updates.append(f"Dataset '{dataset_name}': schema '{old_value}' -> '{new_schema}'")
    
    # Print summary
    if updates:
        print(f"\nUpdated {len(updates)} parameter(s):")
        for update in updates:
            print(f"  - {update}")
    else:
        print("\nNo parameters were updated.")
    
    return dashboard_data

In [0]:
current_user = dbutils.notebook.entry_point.getDbutils().notebook().getContext().userName().get().split('@')[0].replace('.','_')
dbutils.widgets.text('catalog', defaultValue='users')
dbutils.widgets.text('schema', defaultValue=current_user)
dbutils.widgets.text('pre_commit_update', defaultValue='False')

catalog_name = dbutils.widgets.get('catalog')
schema_name = dbutils.widgets.get('schema')
pre_commit_update = dbutils.widgets.get('pre_commit_update').lower() == 'true'

assert catalog_name and schema_name, 'Catalog name and schema name inputs must be provided'

if pre_commit_update:
  print('pre-commit mode, updating values to preconfigured identity values for dash deployment')
  catalog_name = 'home_luis_herrera'
  schema_name = 'pipelineiq'

print(f'Catalog target will be: {catalog_name}')
print(f'Schema target will be: {schema_name}')


In [0]:
dashboard_root_path = '../dashboards/'

all_dashboards = os.listdir(dashboard_root_path)
dashboards_to_update = ['piq_main.lvdash.json', 'piq_sales_managers.lvdash.json']
discovered_dashboards = [f for f in all_dashboards if f in dashboards_to_update]

for dashboard in discovered_dashboards:
  target_file = os.path.join(dashboard_root_path, dashboard)
  print(f"Attempting to update: {target_file}")
  with open(target_file, 'r') as f:
    dashboard_json = json.load(f)
      # Write updated JSON
  updated_json = update_parameters(dashboard_json, catalog_name, schema_name)
  
  with open(target_file, 'w') as f:
      json.dump(updated_json, f, indent=2)
  print(f'Updated dashboard: {target_file} with catalog = {catalog_name}, schema = {schema_name}')



